Student Name: Michel Danjou
Student ID: 18263461

- The 'as' keyword allows you to invoke functionality from the module using an alias for the module name. For example: np.mean() instead of numpy.mean()
- The from keyword allows you to only import the functionality of interest, for example above we import only the PCA class from the sklearn.decomposition module

In [1]:
import numpy as np
import random as rand
import matplotlib.pyplot as plt
from numpy.linalg import eig
from sklearn.decomposition import PCA


As per E-tivity instructions: Use of the matrix class is discouraged, but to allow us to simplify the code slightly, we will use it this week. Its first use will be to store the data that you will perform the PCA transform on. Note that you will likely obtain a higher score if your final version does not use the matrix class.

In [2]:
a_x = 0.05
a_y= 10


In [3]:
#data =  np.matrix([[n*(1+a_x*(rand.random()-0.5)),4*n+ a_y*(rand.random()-0.5)] for n in range(20)])
data =  np.matrix([[n*(1+a_x*(rand.random()-0.5)),4*n+ a_y*(rand.random()-0.5)] for n in range(20)])


The numpy shape property is very useful to get an insight in the dimensions of your data, for example to check whether the features (in this case 2) or the samples (in this case 20) are in the rows or the columns. The notation used here (with columns containing the features and rows containing separate examples) is the standard for Scikitlearn and many other machine learning algorithms.


In [4]:
data.shape


(20, 2)

In [18]:
%reset

import numpy as np
import random as rand
import matplotlib.pyplot as plt
from numpy.linalg import eig
from sklearn.decomposition import PCA

        
class Log:
    DEBUG = 1
    INFO = 2
    ERROR = 3


class My_pca:
    """
    Perform the PCA on a dataset
    
    There is a lot of log statements in this class. I intend to remove
    them in the final code. Leaving them in place for the time being as they
    are useful for debugging. 
    
    BUG ALERT
    =========
    The eigen values calculated by this class match the ones calculated by 
    scikit. 
    
    However, it appears that one of the eigen vectors is the negative version 
    of the one calculted by scikit. Currently investigating the reason behind 
    this.
        
    """

    log_level = Log.ERROR
    nb_components = 2
    eigen_values = []
    eigen_vectors = []


    def __init__(self):
        """ init """


    def __log__(self, message, level=Log.INFO):
        """
        Log a message only if its log level is equal or superior to self.log_level
        """
        if level >= self.log_level:
            print(message)

        
    def fit(self, matrix):
        """ 
        Explicitly using array manipulation instead of 
        easier matrix operations 
        """
        self.matrix = matrix
        
        # Calculate mean values of each column from dataset
        m0 = np.mean(self.matrix[:,0])
        m1 = np.mean(self.matrix[:,1])
        self.__log__(("="*80))
        self.__log__("mean.col0:{} ".format(m0))
        self.__log__("mean.col1:{} ".format(m1))       
        
        # Center the columns by subtracting the corresponding mean
        c0 = matrix[:,0] - m0
        c1 = matrix[:,1] - m1
        self.__log__("c0       :{} ".format(c0), Log.DEBUG)
        self.__log__("c1       :{} ".format(c1), Log.DEBUG)       
        
        # Create a centered matrix 
        c_matrix = np.append(c0, c1, axis=1)
        self.__log__("centered_matrix:\n{}".format(c_matrix), Log.DEBUG)
        
        # Calculate covariance of centered matrix
        my_cov = np.cov(c_matrix, rowvar=False)        
        self.__log__("covariance:\n{}".format(my_cov), Log.DEBUG)
    
        # eigen values, eigen vectors
        eigen_values, eigen_vectors = eig(my_cov)
        self.__log__("eigen_values:\n{}".format(eigen_values))
        self.__log__("eigen_vectors:\n{}".format(eigen_vectors))     
        
        # order eigen values and eigen vectors       
        sorted_eigen_values_indexes = eigen_values.argsort()[::-1]
        sorted_eigen_values = eigen_values[sorted_eigen_values_indexes]
        sorted_eigen_vectors = eigen_vectors[sorted_eigen_values_indexes] 
        self.__log__("sorted_eigen_values_indexes:\n{}".format(sorted_eigen_values_indexes))
        self.__log__("sorted_eigen_values:\n{}".format(sorted_eigen_values))
        self.__log__("sorted_eigen_vectors:\n{}".format(sorted_eigen_vectors))

        # use nb_components to decide how many eigen vectors to keep
        filtered_sorted_eigen_values = sorted_eigen_values[:self.nb_components]
        filtered_sorted_eigen_vectors = sorted_eigen_vectors[:self.nb_components] 
        self.__log__("filtered_sorted_eigen_values:\n{}".format(sorted_eigen_values))
        self.__log__("filtered_sorted_eigen_vectors:\n{}".format(sorted_eigen_vectors))
        
        # calculate projection of dataset onto the eigen vector basis
        #P = eigen_vectors.T.dot(c_matrix.T)        
        #self.__log__("projected  :\n{}".format(P.T), Log.DEBUG)
        
        # save results as class variables
        self.eigen_values = filtered_sorted_eigen_values
        self.eigen_vectors = filtered_sorted_eigen_vectors
            
    
    def transform(self, data):
        """
        Calculate projection of dataset onto the eigen vector basis
        
        BUG:
        This method does work yet...
        """
        self.fit(data)
        self.projection = self.eigen_vectors.T.dot(data.T)
        
        self.__log__("eigen_values:\n{}".format(self.eigen_values), Log.DEBUG)
        self.__log__("eigen_vectors:\n{}".format(self.eigen_vectors), Log.DEBUG)
        self.__log__("projected  :\n{}".format(self.projection), Log.DEBUG)
        
        
        
def build_dataset():
    """
    Create a dataset
    """
    a_x = 0.05
    a_y= 10

    data =  np.matrix([[n*(1+a_x*(rand.random()-0.5)),4*n+ a_y*(rand.random()-0.5)] for n in range(20)])

    print(data)
    print(data.shape)
    return data


def scikit_pca( matrix, nb_components):
    """
    Calculate the PCA using Scikit APIs
    """
    pca = PCA(nb_components)
    pca.fit(matrix)

    return pca

      
def test():

    my_pca = My_pca()

    # Calculate PCA using scikit, nb_components=2
    pca = scikit_pca(data, 2)
    print()
    print("scikit.pca with nb_components=2")
    print("eigen_values:", pca.explained_variance_)
    print("eigen_vectors:", pca.components_)
    print("transformed_data:", pca.transform(data))

    # Calculate PCA using scikit, nb_components=1
    pca = scikit_pca(data, 1)
    print()
    print("scikit.pca with nb_components=1")
    print("eigen_values:", pca.explained_variance_)
    print("eigen_vectors:", pca.components_)
    print("transformed_data:", pca.transform(data))

    # Calculate PCA using homebrew code, nb_components=2
    my_pca.nb_components=2
    my_pca.fit(data)
    print()
    print("pca homebrew nb_components=", my_pca.nb_components)
    print("eigen_values:", my_pca.eigen_values)
    print("eigen_vectors:", my_pca.eigen_vectors)  
        
    # Calculate PCA using homebrew code, nb_components=1
    my_pca.nb_components=1
    my_pca.fit(data)
    print()
    print("pca homebrew nb_components=", my_pca.nb_components)
    print("eigen_values:", my_pca.eigen_values)
    print("eigen_vectors:", my_pca.eigen_vectors)
    
    # Calculate transformation using homebrew code, nb_coomponent=2
    my_pca.nb_components=2
    my_pca.transform(data)
    print()
    print("Transform using homebrew code, nb_components=", my_pca.nb_components)
    print("Transform:", my_pca.projection)
    
    # Calculate transformation using homebrew code, nb_coomponent=1
    my_pca.nb_components=1
    #my_pca.transform(data)
    #print()
    #print("Transform using homebrew code, nb_components=", my_pca.nb_components)
    #print("Transform:", my_pca.projection)
    

data = build_dataset()    
test()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
[[ 0.          4.14656817]
 [ 1.02310036  8.51322675]
 [ 2.01239035  9.15115242]
 [ 3.00225114  9.6606455 ]
 [ 4.09852886 19.29371226]
 [ 5.00357848 22.12927107]
 [ 6.09978631 27.61125185]
 [ 6.8843758  24.60789524]
 [ 7.98449106 34.80309688]
 [ 8.77962992 31.78479252]
 [10.02214812 38.6168585 ]
 [11.1472456  45.89093366]
 [12.01691203 52.79079839]
 [12.73588537 48.02836754]
 [14.27944104 51.83876991]
 [15.05898697 55.45093017]
 [16.12890754 63.98479441]
 [17.22592229 69.06149794]
 [18.44831318 70.79169443]
 [19.36691738 71.14046299]]
(20, 2)

scikit.pca with nb_components=2
eigen_values: [5.24002641e+02 5.21638274e-01]
eigen_vectors: [[-0.26062686 -0.96543961]
 [-0.96543961  0.26062686]]
transformed_data: [[ 3.51426364e+01  4.21388990e-01]
 [ 3.06602438e+01  5.71715893e-01]
 [ 2.97865295e+01 -2.17123282e-01]
 [ 2.90366604e+01 -1.03998652e+00]
 [ 1.94507968e+01  4.12259484e-01]
 [ 1.64773558e+01  2.77511527e-01]
 [ 1.08991

# Comparison between Scikit and homebrew PCA
## Scikit

  * There are 2 eigen values: **580** and **0.45**. 
  * Because the 2nd eigen value is low we can deduct that the second eigen vector will not carry significant information and can be ignored.
  * Looking at the values of the eigen vectors we can clearly see that they are **orthogonal**
  * The first eigen vector is **[-0.24191617 -0.97029715]**

```python
scikit.pca with nb_components=2
eigen_values: [5.80787811e+02 4.56934586e-01]
eigen_vectors: [[-0.24191617 -0.97029715]
 [-0.97029715  0.24191617]]
```
## Homebrew implementation
  * There are 2 eigen values: **580** and **0.45**. 
  * Because the 2nd eigen value is low we can deduct that the second eigen vector will not carry significant information and can be ignored.
  * Looking at the values of the eigen vectors we can clearly see that they are **orthogonal**
  * The first eigen vector is **[ 0.24191617 -0.97029715]**
  
```python
pca homebrew nb_components= 2
eigen_values: [5.80787811e+02 4.56934586e-01]
eigen_vectors: [[ 0.24191617 -0.97029715]
 [-0.97029715 -0.24191617]]
```
`I note sign discrepencies between the eigen vectors returned by Scikit and the Homebrew implementation. The eigen basis is therefore different. I am looking for an explanation.`


In [ ]:
# Calculate PCA using scikit, nb_components=2
pca = scikit_pca(data, 2)
print()
print("scikit.pca with nb_components=2")
print("eigen_values:", pca.explained_variance_)
print("eigen_vectors:", pca.components_)

# Calculate PCA using scikit, nb_components=1
pca = scikit_pca(data, 1)
print()
print("scikit.pca with nb_components=1")
print("eigen_values:", pca.explained_variance_)
print("eigen_vectors:", pca.components_)



# Comparison between nb_component=1 and nb_component=2

Since the value of the second eigen vector low, the second eigen vector can be ignored and thus the resulting eigen space can be simplified to **one dimension**.